In [ ]:
# @title 데이터취득
# https://drive.google.com/file/d/1iS1F7R1waTTOmL7Ab9i2ULQTqj8tIhee/view?usp=sharing

import gdown, os, zipfile

file_id = '1iS1F7R1waTTOmL7Ab9i2ULQTqj8tIhee'
output = 'data.zip'

gdown.download(f'http://drive.google.com/uc?id={file_id}', output, quiet=False)

output_dir = 'furniture'
os.makedirs(output_dir, exist_ok=True)  # 폴더가 없으면 생성

with zipfile.ZipFile(output, 'r') as z:
    z.extractall(output_dir)  # output_dir에 압축 해제
    print(f"압축 해제 완료: {output_dir}")

# 압축 해제된 폴더 내용 확인
print("압축 해제된 파일 및 폴더 목록:")
print(os.listdir(output_dir))


Downloading...
From (original): http://drive.google.com/uc?id=1iS1F7R1waTTOmL7Ab9i2ULQTqj8tIhee
From (redirected): https://drive.google.com/uc?id=1iS1F7R1waTTOmL7Ab9i2ULQTqj8tIhee&confirm=t&uuid=8b5f7cc2-fd0a-4d36-bf06-df11d2918ca6
To: /content/data.zip
100%|██████████| 557M/557M [00:11<00:00, 48.8MB/s]


압축 해제 완료: furniture
압축 해제된 파일 및 폴더 목록:
['data']


In [ ]:
# @title Sequence 객체 만들기
from tensorflow.keras.utils import Sequence
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.utils import shuffle

class FurnitureSequence(Sequence):
  def __init__(self, images, labels, batch_size=32, image_size=224, augmentor=None, preprocess_function=None, shuffle=True):
    self.images = images
    self.labels = labels
    self.batch_size = batch_size
    self.image_size = image_size
    self.augmentor = augmentor
    self.preprocess_function = preprocess_function
    self.shuffle = shuffle
    self.indexes = np.arange(len(images))
    if self.shuffle:
      np.random.shuffle(self.indexes)

  def __len__(self):
    return int(np.ceil(len(self.images) / self.batch_size)) # 배치 사이즈만큼 돌리기

  def __getitem__(self, index): #index번재 배치 데이터 반환
    start = index * self.batch_size
    end = (index + 1) * self.batch_size
    this_batch_images = self.images[start:end]
    batch_labels = self.labels[start:end] if self.labels is not None else None # 예측에는 라벨 필요없음

    # 배치 크기에 맞는 배열 초기화 (리사이즈된 결과를 저장할 새로운 배열 필요)
    batch_images = np.zeros((this_batch_images.shape[0], self.image_size, self.image_size, 3), dtype=np.float32)

    for i in range(this_batch_images.shape[0]):
      image = this_batch_images[i]

      # 데이터 증강 적용
      if self.augmentor is not None:
        image = self.augmentor(image)['image'] # augment 딕셔너리 반환해서, 필요한 데이터(이미지만 추출)만 명시적으로 추출, 데이터 증강 도구는 이미지 외의 데이터를 함께 처리할 수 있어서('mask','bboxes' 등)

      # 리사이즈
      image = cv2.resize(image,(self.image_size, self.image_size))

      # 전처리
      if self.preprocess_function is not None:
        image = self.preprocess_function(image)

      batch_images[i] = image

    return (batch_images, batch_labels) if self.labels is not None else batch_images

    def on_epoch_end(self): # 에포크 종료 후 데이터 섞기.
        if self.shuffle:
            self.images, self.labels = shuffle(self.images, self.labels)





In [ ]:
# @title 데이터 준비

from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# 이미지와 레이블 준비 (예: NumPy 배열)
images = np.random.rand(1000, 256, 256, 3)  # 가상 이미지 데이터
labels = np.random.randint(0, 4, size=(1000,))  # 4개의 클래스

# One-Hot Encoding
from tensorflow.keras.utils import to_categorical
labels = to_categorical(labels, num_classes=4)


In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.resnet50 import preprocess_input

# 데이터 분할
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.3, random_state=42)
tr_images, val_images, tr_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# Sequence 객체 생성
train_seq = FurnitureSequence(tr_images, tr_labels, batch_size=32, image_size=224, preprocess_function=preprocess_input, shuffle=True)
val_seq = FurnitureSequence(val_images, val_labels, batch_size=32, image_size=224, preprocess_function=preprocess_input, shuffle=False)
test_seq = FurnitureSequence(test_images, test_labels, batch_size=32, image_size=224, preprocess_function=preprocess_input, shuffle=False)

tr_batch_images, tr_batch_labels = next(iter(train_seq)) # seq객체 next로 보고싶으면 iter객체로 감싸서 볼 수 있고, 배치 이미지를 반환함
tr_batch_images.shape, tr_batch_labels.shape

((32, 224, 224, 3), (32, 4))

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


In [ ]:
# @title 모델 생성

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# ResNet50 기본 모델 로드 (ImageNet 가중치 사용)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 새로운 Fully Connected Layer 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global Average Pooling
x = Dense(1024, activation='relu')(x)  # Fully Connected Layer
predictions = Dense(4, activation='softmax')(x)  # 최종 출력층 (4개의 클래스)

# 최종 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# ResNet50의 기존 층 동결
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# @title 모델 컴파일

from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 콜백설정
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

reduce_lr_on_plateau_cb =  ReduceLROnPlateau(patience=3, factor=0.5, verbose=1)
early_stop = EarlyStopping(monitor='val_accuracy', patience=5, mode='max', restore_best_weights=True)
callbacks = [reduce_lr_on_plateau_cb, early_stop]


In [ ]:
# @ tutle 모델 학습
history = model.fit(
    train_seq,
    validation_data=val_seq,
    epochs=30,
    callbacks=callbacks
)


Epoch 1/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 19s 532ms/step - accuracy: 0.2374 - loss: 1.7965 - val_accuracy: 0.2786 - val_loss: 1.4564 - learning_rate: 0.0010
Epoch 2/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 10s 110ms/step - accuracy: 0.2411 - loss: 1.5405 - val_accuracy: 0.2000 - val_loss: 1.5497 - learning_rate: 0.0010
Epoch 3/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 123ms/step - accuracy: 0.2360 - loss: 1.5334 - val_accuracy: 0.2000 - val_loss: 1.5021 - learning_rate: 0.0010
Epoch 4/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.2433 - loss: 1.4836
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 122ms/step - accuracy: 0.2432 - loss: 1.4837 - val_accuracy: 0.2214 - val_loss: 1.4636 - learning_rate: 0.0010
Epoch 5/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 109ms/step - accuracy: 0.2647 - loss: 1.4641 - val_accuracy: 0.2214 - val_loss: 1.4185 - learning_rate: 5.0000e-04
Epoch 6/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 125ms/step - accuracy: 0.2296 - loss: 1.414

In [ ]:
# @ title 동결 해제 후 미세 조정(fine-Tuning)

# ResNet50의 상위 20개 층 동결 해제
for layer in base_model.layers[-20:]:
    layer.trainable = True

# 모델 재컴파일 (학습률을 낮춤)
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fine-Tuning 학습
history_fine = model.fit(
    train_seq,
    validation_data=val_seq,
    epochs=30,
    callbacks=callbacks
)


Epoch 1/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 27s 777ms/step - accuracy: 0.2518 - loss: 1.4122 - val_accuracy: 0.2786 - val_loss: 1.4409 - learning_rate: 1.0000e-05
Epoch 2/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 127ms/step - accuracy: 0.2955 - loss: 1.3839 - val_accuracy: 0.2786 - val_loss: 1.4355 - learning_rate: 1.0000e-05
Epoch 3/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 127ms/step - accuracy: 0.3084 - loss: 1.3677 - val_accuracy: 0.2786 - val_loss: 1.4298 - learning_rate: 1.0000e-05
Epoch 4/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 153ms/step - accuracy: 0.3360 - loss: 1.3625 - val_accuracy: 0.2786 - val_loss: 1.4154 - learning_rate: 1.0000e-05
Epoch 5/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 127ms/step - accuracy: 0.4083 - loss: 1.3494 - val_accuracy: 0.2786 - val_loss: 1.4046 - learning_rate: 1.0000e-05


In [ ]:
# @title 모델 평가
loss, accuracy = model.evaluate(test_seq)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - accuracy: 0.2969 - loss: 1.4187
Test Accuracy: 26.33%


In [ ]:
# @title 테스트 이미지 예측

from tensorflow.keras.preprocessing import image
import numpy as np

# 테스트 이미지 로드
img_path = '/content/의자.jpeg'  # 테스트 이미지 경로
img = image.load_img(img_path, target_size=(224, 224))  # ResNet 입력 크기에 맞게 리사이즈
img_array = image.img_to_array(img) / 255.0  # 정규화
img_array = np.expand_dims(img_array, axis=0)  # 배치 차원 추가

# 예측 수행
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)
class_labels = ['밥상', '의자', '소파', '서랍장']

print(f"Predicted Class: {class_labels[predicted_class]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Predicted Class: 서랍장
